In [ ]:
import torch
import torch.nn as nn
class DQN(nn.Module):
  

In [ ]:
import diambraArena
from diambraArena.gymUtils import showGymObs

settings = {}
settings["romsPath"] = "/home/nabil/Downloads/"
settings["gameId"] = "doapp"
settings["actionSpace"] = "discrete"
settings["attackButCombination"] = False # reduce action space size

envId = "TestEnv"
env = diambraArena.make(envId, settings)
observation = env.reset()
showGymObs(observation, env.charNames)

deep q net stuff


https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html

[drawing upon architecture used here for feature extraction](https://www.youtube.com/watch?v=ynlNIfFl_9c)

In [ ]:
import math
import torch
import torch.nn as nn
import cv2


class ImageNetwork(nn.Module):
    # TO DO change architecutre ; input size
    # image transforms
    def __init__(self, image_width=480, image_height=512, num_actions = 16):
        super(DQN, self).__init__()
        self.conv_relu_stack = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 32, kernel_size=5, stride=2),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, kernel_size = 5, stride=2),
            nn.BatchNorm2d(32)
            )
            
        def output_shape(size, kernel_size=5, stride=2):
            return math.floor(((size - (kernel_size - 1) - 1)/stride) + 1)
            
        convw = output_shape(output_shape(output_shape(image_width)))
        convh = output_shape(output_shape(output_shape(image_height)))
        
        self.get_actions = nn.Linear(convw*convh*32, num_actions)
        
	def forward(self, x):
		x = state
		if not isinstance(x, torch.Tensor):
		x = torch.tensor(x)
		x = self.linear_relu_stack(x)
		x = get_actions(x)
		return x
		
class PlayerObservationsNet(nn.Module):
    """
    Input: P1/P2 Health, Stage, P1/P2 Side
    """
	def __init__(self, input_dim, output_dim):
		super(PlayerObservationsNet, self).__init__()
		self.linear_relu_stack = nn.Sequential(
			nn.Linear(input_dim, 32),
			nn.ReLU(),
			nn.Linear(32,32),
			nn.ReLU(),
			nn.Linear(32,output_dim)
		)
	def forward(self, x):
		x = self.linear_relu_stack(x)
		return x
		
class GetActions(nn.Module):
	"""
	Input: outputs of the image and player-observations networks
	"""
	def


def player_obs_to_vec(observations):
    """
    Process observations returned by env into a flat vector that can
    be used by the PlayerObservationsNet
    """
    
n_actions = env.actions_space.n               
image_net = ImageNetwork(num_actions = n_actions).to(device)

optimizer = optim.RMSprop(policy_net.parameters())

def predict(x):
  """
  returns index (representing action) with greatest Q-value
  """
  return torch.argmax(nn.functional.softmax(x,dim=0))
    
def loss_fcn(y, Q):
    return nn.functional.mse_loss(y, Q)#.mean()

opt = torch.optim.RMSprop(model.parameters(),lr=0.001)

def update_weights(X, Y):
    '''
    input: nparray of indices of replay memory D to use as training samples
    This updates Q Network parameters via gradient descent step
    '''
    opt.zero_grad()
    loss = loss_fcn(X, Y)
    loss.backward()
    opt.step()
 

**gameplay loop**

In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

def select_action(epsilon, observation):
  """
  Input: epsilon - probability of choosing a random action
  Output: Pick action according to probability below

  P(pick random action) = epsilon
  P(pick action with best Q value) = 1 - epsilon
  """
    action_type = np.random.choice([0,1],p=[epsilon,1-epsilon])
    if action_type == 0: # do random action
        action = env.action_space.sample()
    else: # do action with greatest Q-value
        action = int(predict(theDeepQNet(observation))) # need to cast tensor to int

memory = ReplayMemory(capacity=10000)
epsilon = 0.9
min_epsilon = 0.05
decay_rate = 0.995 # for epsilon
batch_size = 32
gamma = 0.999

theDeepQNet = NEURAL_NETWORK_THAT_GIVES_US_THE_PREDICTION_FOR_ACTIONS()
state = observation
while True:

    epsilon = max(epsilon*decay_rate, min_epsilon)
    
    # need to preprocess the observation data before inputting to model
    player_inp = player_obs_to_vec(observation) # preprocessed input
    img_inp = img_obs_to_tensor(observation) # preprocessed input
    input_for_model = CombinedNetwork(ImageNetwork(img_inp), PlayerNetwork(player_inp))

    # change var name for model
    predicted_move = thedeepQnet(input_for_model)
    predicted_move = select_action(epsilon) 

    next_state, reward, done, info = env.step(predicted_move)
    memory.push(state, action, reward, next_state)
    state = next_state

    train_indices = np.random.choice(np.arange(len(memory)),size=batch_size)
    train_samples = [memory[i] for i in train_indices]

    Q_values = [model(memory[i][0])[0][memory[i][1]] for i in train_indices] # list of tensors
    expected_Q_values = [memory[i][2] + gamma*max(model(memory[i][3])[0]) for i in train_indices] # list of tensors
    Q_values, expected_Q_values = torch.stack(Q_values), torch.stack(expected_Q_values)
    update_weights(Q_values, expected_Q_values)

    showGymObs(observation, env.charNames)
    print("Reward: {}".format(reward))
    print("Done: {}".format(done))
    print("Info: {}".format(info))

    if done:
        observation = env.reset()
        showGymObs(observation, env.charNames)
        break

env.close()